# get page URL

In [1]:
all_categories=[]
num=[8,4,11,5,2,13,4,0,0,0]
for i in range(10):
    for j in range(num[i]+1):
        if num[i]!=0:
            all_categories.append(40000+100*(i+1)+(j+1))
        num[i]-=1
        
        if num[i]==-1:
            all_categories.append(400+(i+1))
        if num[i]==0:
            break

url_list=[]
for categories in all_categories:
    for pages in range(99):
        url_list.append("https://kin.naver.com/qna/kinupList.naver?dirId="+str(categories)+"&page="+str(pages+1))
print(len(url_list))

4950


# get all URL

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import time

start_time=time.time()
side_url=[]

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    link_tags = soup.find_all('a')
    for link_tag in link_tags:
        href = link_tag.get('href')
        absolute_url = urljoin(url, href)
        if "https://kin.naver.com/qna/detail.naver?d1id=4&" in absolute_url:
            side_url.append(absolute_url)
            
df=pd.DataFrame(side_url)
df.to_excel("./Naver_URL.xlsx")
print(time.time()-start_time)

1087.7369277477264


# data collect

In [5]:
side_url = list(set(side_url))
print(len(side_url))

72708


In [6]:
result_dict_list=[]

In [7]:
start_time=time.time()
for url in side_url:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    result_dict = {
        "quest": [], #제목
        "detail": [], #내용
        "first_answer": [], #채택순으로 fisrst, second, ..
        "first_answer_accepted" : [], #답변자의 채택답변수, 받은 감사수
        "second_answer": [],
        "second_answer_accepted": [],
        "third_answer" : [],
        "third_answer_accepted": [],
        "fourth_answer" : [],
        "fourth_answer_accepted": [],
        "views" : [],
        "url" : []
    }
    
    ques_divs=soup.find("div", class_="title")
    detail_divs=soup.find("div", class_="c-heading__content")
    answer_divs = soup.find_all("div", class_="_endContentsText c-heading-answer__content-user")
    answer_accepted_divs = soup.find_all("div", class_="item_list")
    views_divs = soup.find_all("span", class_='c-userinfo__info')[1].text.split()[1]    
    
    if ques_divs is not None:
        ques_text = ques_divs.get_text(strip=True)
    else:
        ques_text = None
    
    if detail_divs is not None:
        detail_text = detail_divs.get_text(strip=True)
    else:
        detail_text = None
        
    count_answer=0
    
    for answer_div,answer_accepted_div in zip(answer_divs,answer_accepted_divs):
        if count_answer==0:
            first_text=False
            first_accepted_text=False
            second_text=False
            second_accepted_text=False
            third_text=False
            third_accepted_text=False
            fourth_text=False
            fourth_accepted_text=False
            
            first_text = answer_div.get_text(strip=True)
            first_accepted_text = answer_accepted_div.get_text(strip=True)
        elif count_answer==1:
            second_text = answer_div.get_text(strip=True)
            second_accepted_text = answer_accepted_div.get_text(strip=True)
        elif count_answer==2:
            third_text = answer_div.get_text(strip=True)
            third_accepted_text = answer_accepted_div.get_text(strip=True)
        elif count_answer==3:
            fourth_text = answer_div.get_text(strip=True)
            fourth_accepted_text = answer_accepted_div.get_text(strip=True)
        count_answer+=1

    
    if ques_text:
        result_dict["quest"].append(ques_text)
    else:
        result_dict["quest"].append(None)
    
    if detail_text:
        result_dict["detail"].append(detail_text)
    else:
        result_dict["detail"].append(None)
    
    if first_text:
        result_dict["first_answer"].append(first_text)
        result_dict["first_answer_accepted"].append(first_accepted_text)
    else:
        result_dict["first_answer"].append(None)
        result_dict["first_answer_accepted"].append(None)
        
    if second_text:
        result_dict["second_answer"].append(second_text)
        result_dict["second_answer_accepted"].append(second_accepted_text)
    else:
        result_dict["second_answer"].append(None)
        result_dict["second_answer_accepted"].append(None)

    if third_text:
        result_dict["third_answer"].append(third_text)
        result_dict["third_answer_accepted"].append(third_accepted_text)
    else:
        result_dict["third_answer"].append(None)
        result_dict["third_answer_accepted"].append(None)
        
    if fourth_text:
        result_dict["fourth_answer"].append(fourth_text)
        result_dict["fourth_answer_accepted"].append(fourth_accepted_text)
    else:
        result_dict["fourth_answer"].append(None)
        result_dict["fourth_answer_accepted"].append(None)
    
    result_dict["url"].append(url)
    result_dict["views"].append(views_divs)
    result_dict_list.append(result_dict)
print(time.time()-start_time)

IndexError: list index out of range

In [14]:
print(len(result_dict_list))

67503


In [9]:
import pandas as pd

df=pd.DataFrame(result_dict_list)
df.to_excel("./Naver_economy_all_data1.xlsx")